In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
datapath = 'E:/Dissertation Data'
#datapath = 'C:/Users/'+os.getlogin()+'/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\User\\Google Drive\\University\\Dissertation\\Code'

In [2]:
import pickle
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

#from glove import Corpus, Glove

from gensim.models import Word2Vec

In [3]:
from nltk.metrics import (
    BigramAssocMeasures,
    TrigramAssocMeasures,
    NgramAssocMeasures,
)

from nltk.metrics.spearman import (
    spearman_correlation,
    ranks_from_scores,
)

We then automatically score the million most strongly associated
n-grams (i.e., roughly the top 5% of the Poisson-ranked
list) for compositionality.

Using word2vec (Mikolov et al., 2013) with the parameters
found to be most effective by Baroni et al. (2014), we
build a word embedding vector for every simplex word in
the vocabulary (ca. 1M types), as well as for each MWE candidate.

* Continuous bag of words model with 400-dimensional vectors, window size 5, subsampling with t = 10^-5, negative sampling with 10 samples. We build vectors only for tokens observed 20 times or more in the corpus.

We then compute the cosine similarity of the vector
representation for a MWE candidate with the vectors of its
constituent words, and take the arithmetic mean. 
In scoring
the compositionality of a candidate, we do not measure the
cosine similarity of the MWE with any stop words it may
contain, as stop words may be assumed to be semantically
uninformative.
* Stop words are taken here to be the 50 most frequent words in the vocabulary.

In [4]:
# Stopwords from corpus - 50 most frequent

with open(datapath+'/Corpora/wiki/enwiki_20200520/Tagged/stop_clean.pkl', 'rb') as pfile:
    stop = pickle.load(pfile)

In [5]:
stop

{'0|CD',
 '1|CD',
 '2|CD',
 'A|DT',
 'He|PRP',
 'In|IN',
 'It|PRP',
 'New|NNP',
 'The|DT',
 'after|IN',
 'also|RB',
 'and|CC',
 'an|DT',
 'are|VBP',
 'as|IN',
 'at|IN',
 'a|DT',
 'been|VBN',
 'be|VB',
 'but|CC',
 'by|IN',
 'first|JJ',
 'for|IN',
 'from|IN',
 'had|VBD',
 'has|VBZ',
 'he|PRP',
 'his|PRP$',
 'in|IN',
 'is|VBZ',
 'its|PRP$',
 'it|PRP',
 'not|RB',
 'of|IN',
 'one|CD',
 'on|IN',
 'or|CC',
 'that|IN',
 'that|WDT',
 'their|PRP$',
 'the|DT',
 'this|DT',
 'to|IN',
 'to|TO',
 'two|CD',
 'was|VBD',
 'were|VBD',
 'which|WDT',
 'who|WP',
 'with|IN'}

In [6]:
ngram_eval = pd.read_pickle(datapath+'/Corpora/wiki/enwiki_20200520/Tagged/ngram_eval_clean.pkl')

ngram_eval

,ngram,freq,poisson,len,batch
0,"(Dechawat|NNP, Poomjaeng|NNP)",20,-666.206003,2,1
1,"(MSC1|NN, MSC2|NN)",20,-666.206003,2,1
2,"(resting_place_coordinates|NNS, burial_place|VBP)",20,-666.206003,2,1
3,"(Neophron|NNP, percnopterus|NNP)",20,-667.613790,2,1
4,"(Eugeniya|NNP, Pashkova|NNP)",20,-668.956074,2,1
...,...,...,...,...,...
499995,"(Mom|NNP, Deserves|VBZ, a|DT)",28,-1711.315270,3,4
499996,"(that|IN, he|PRP, takes|VBZ)",22,-1711.316813,3,2
499997,"(got|VBD, the|DT, best|JJS)",22,-1711.316966,3,6
499998,"(In|IN, his|PRP$, novel|JJ)",22,-1711.317012,3,1


In [7]:
min_freq = 20
eval_count = 500000

batch_count = max(ngram_eval.batch)

In [8]:
ngram_eval.batch.value_counts()

 1     179196
 2     122020
 3      79022
 4      51014
 5      29281
 6      17771
 7       9368
 8       5678
 9       2898
-1       2017
 10      1561
-2        174
Name: batch, dtype: int64

In [9]:
from nltk.tokenize import MWETokenizer
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Flatten down to a single number
def cosim(x,y):
    return cosine_similarity(x.reshape(1,-1), y.reshape(1,-1))[0][0]

In [10]:
def mwe_score(exp, model, stats_frame):
    
    # Combined token for MWE
    mwetoken = '+'.join(exp)

    # Stopwords - 1 if component is a stopword, 0 if present, -1 if simplex word missing from vocab, -2 if MWE missing
    sws = []
    # Component vectors
    cvs = []

    #  Neighbours in original & MWE-aware space
    #oldn = []
    #newn = []

    # List of individual word similarities (where present in the vocab)
    css = []

    # Empty array
    earr = np.empty(400)
    earr[:] = np.nan

    # Check that combined token exists in the vocab. This protects against inflation of n-gram counts caused by repeats
    #  of the same token (e.g. in lists like https://simple.wikipedia.org/wiki/List_of_cities,_towns_and_villages_in_Fars_Province)
    if mwetoken in batch_model.wv.vocab:

        mwv = model.wv[mwetoken]

        for w in exp:
            if w in model.wv.vocab:
                cvs.append(model.wv[w])

                #oldn.append(batch_model.wv.most_similar(w, topn=5))

                if w in stop:
                    sws.append(1)
                    css.append(np.nan)
                else:
                    sws.append(0)
                    css.append(cosim(model.wv[w], mwv ))

            # If component is absent from vocab
            else:
                sws.append(-1)
                cvs.append(earr)
                css.append(np.nan)

                #oldn.append([])

        #  Mean cosim
        if min(sws) >= 0:
            cs = np.nanmean(css)
        else:
            cs = np.nan

        #newn = batch_model.wv.most_similar(mwetoken, topn=5)

    # Combined token missing from vocab - mark with defaults
    else:
        sws = [-2]
        mwv = np.empty(400)
        mwv[:] = np.nan
        cs = np.nan


    # Append to stats df
    return stats_frame.append({
        'ngram'  : exp,
        'stopwords' : sws,
        'mwe_vector' : mwv,
        'component_vectors' : cvs,
        'component_cosims'  : css,
        'cosine_sim'  : cs,
        #'base_nearest': oldn,
        #'mwe_nearest' : newn,
    }, ignore_index=True)

In [11]:
%%time

batch_dfs = {}

for bb in range(batch_count):
    print('Processing batch {} of {}'.format(bb+1,batch_count))
    
    # Subset DataFrame
    batch_dfs[bb] = ngram_eval[ngram_eval.batch == bb+1].reset_index(drop=True)
    
    # Initialise MWETokenizer
    batch_token_mwe = MWETokenizer(list(batch_dfs[bb].ngram) , separator='+')
    
    # Load model
    print('Loading w2v model')

    batch_model = Word2Vec.load(datapath+'/Models/1 w2v/Tagged/w10p_tagged_clean_batch{}.model'.format(bb+1))
    
    print('Gathering MWE stats')
    
    # For each MWE, evaluate stats. Record vectors (in case we want to calculate different metrics later).
    statsf = pd.DataFrame(columns=['ngram', 'stopwords', 'mwe_vector', 'component_vectors', 'component_cosims', 
                                   'cosine_sim']) # , 'base_nearest', 'mwe_nearest'
    batch_len = len(batch_dfs[bb].ngram)
    if batch_len >= 20000: 
        printer = 5000
    elif batch_len >= 5000: 
        printer = 1000
    else:
        printer = 200
        
    _i = 0
    
    for exp in batch_dfs[bb].ngram:
        if _i % printer == 0:
            print(' MWE '+str(_i)+'/'+str(batch_len)+': '+'+'.join(exp))
        _i += 1
        
        statsf = mwe_score(exp, batch_model, statsf)
        
    #  Join back onto DataFrame
    batch_dfs[bb] = batch_dfs[bb].merge(statsf, on='ngram')
    

Processing batch 1 of 10
Loading w2v model
Gathering MWE stats
 MWE 0/179196: Dechawat|NNP+Poomjaeng|NNP
 MWE 5000/179196: ta|NNP+Know|NNP
 MWE 10000/179196: Ub|NNP+Iwerks|NNP
 MWE 15000/179196: Joseph|NNP+Merrick|NNP
 MWE 20000/179196: Scouts|NNS'+Association|NNP
 MWE 25000/179196: newspapers|NNS+ran|VBD
 MWE 30000/179196: a|DT+crevice|NN
 MWE 35000/179196: The|DT+GeForce|NNP
 MWE 40000/179196: Tajik|NNP+Cup|NNP
 MWE 45000/179196: 2014|CD+Skate|NNP
 MWE 50000/179196: US|NNP+-24|CD
 MWE 55000/179196: well|RB+again|RB
 MWE 60000/179196: to|IN+resolution|NN
 MWE 65000/179196: Mark|NNP+Casstevens|NNP
 MWE 70000/179196: attended|VBD+UCLA|NNP
 MWE 75000/179196: of|IN+disuse|NN
 MWE 80000/179196: C|NN-sharp|JJ+minor|JJ+Op|NN
 MWE 85000/179196: inheritance|NN+tax|NN
 MWE 90000/179196: often|RB+remembered|VBN
 MWE 95000/179196: multiple|JJ+targets|NNS
 MWE 100000/179196: intelligence|NN+test|NN
 MWE 105000/179196: in|IN+1986
 MWE 110000/179196: great|JJ+houses|NNS
 MWE 115000/179196: s|POS+Yea

In [13]:
# Merge dataframes, sort by compositionality metric, export

# Also want the default batches with batch no < 0
all_batches = ngram_eval[ngram_eval.batch < 0]

for d in range(batch_count):
    print('Appending batch {} of '.format(d+1)+str(batch_count))
    all_batches = all_batches.append(batch_dfs[d])
    
all_batches = all_batches.sort_values('cosine_sim')
all_batches = all_batches.reset_index(drop=True)

all_batches.to_csv(datapath+'/Models/1 w2v/Tagged/Results/w10p_tagged_clean_output_001.csv', index=False)

Appending batch 1 of 10
Appending batch 2 of 10
Appending batch 3 of 10
Appending batch 4 of 10
Appending batch 5 of 10
Appending batch 6 of 10
Appending batch 7 of 10
Appending batch 8 of 10
Appending batch 9 of 10
Appending batch 10 of 10


In [14]:
all_batches

,ngram,freq,poisson,len,batch,stopwords,mwe_vector,component_vectors,component_cosims,cosine_sim
0,"(However|RB, since|IN, then|RB)",20,-1501.356357,3,4,"[0, 0, 0]","[-0.08021507, 0.029799368, -0.11063751, 0.0798...","[[0.44135073, -0.1859463, 2.1679575, -1.754787...","[-0.22124107, -0.1947155, -0.2786541]",-0.231537
1,"(has|VBZ, been|VBN, following|VBG)",20,-1563.921228,3,4,"[1, 1, 0]","[-0.06269902, -0.0087266695, -0.061697353, 0.0...","[[0.8249299, -1.3578229, 0.97652656, -0.036999...","[nan, nan, -0.22919384]",-0.229194
2,"(both|CC, times|NNS, by|IN)",21,-1608.202878,3,8,"[0, 0, 1]","[-0.07510896, 0.04154765, -0.007869523, 0.1447...","[[1.2414573, -0.1210257, 0.77684075, -1.404658...","[-0.26926857, -0.18902278, nan]",-0.229146
3,"(after|IN, both|CC, the|DT)",20,-1643.568467,3,3,"[1, 0, 1]","[-0.033339113, -0.044894636, 0.00069739716, -0...","[[0.95313185, -0.647143, 0.3539722, -3.1936626...","[nan, -0.22798495, nan]",-0.227985
4,"(However|RB, even|RB, after|IN)",20,-1499.074445,3,2,"[0, 0, 1]","[-0.058937408, 0.0068662553, -0.00825334, 0.13...","[[-0.098684065, 0.9381516, 0.47127044, -1.7793...","[-0.17959411, -0.27630407, nan]",-0.227949
...,...,...,...,...,...,...,...,...,...,...
499995,"(:|:, Paleogene|NNP, from|IN)",24,-1669.762522,3,10,[-2],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[],NaN
499996,"(of|IN, $|$, 3,000|CD)",22,-1681.886235,3,10,[-2],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[],NaN
499997,"(la|NNP, la|NNP, la|NNP)",25,-1687.517191,3,10,[-2],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[],NaN
499998,"(2015|CD, -2016|CD, school|NN)",25,-1696.555611,3,10,[-2],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[],NaN


In [ ]:
#type(all_batches.stopwords[0])

In [15]:
# RuntimeWarning: Mean of empty slice
#  Appear to be taking the np.nanmean() of a list of NaNs

def inval(inlist,val=0):
    if type(inlist) != list:
        return False
    return val in inlist

#nozero = all_batches[~all_batches.stopwords.apply(inval)]

#minone = nozero[nozero.stopwords.apply(inval,val=-1)]

#minone[minone.stopwords.apply(inval,val=1)]

In [16]:
w10p_res_light = all_batches.drop(columns=['mwe_vector', 'component_vectors'])

In [17]:
w10p_res_light.to_csv(datapath+'/Models/1 w2v/Tagged/Results/w10p_tagged_clean_light_001.csv', index=False)